In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

inp = pd.read_excel("~//Downloads//Min_wage.xlsx")
inp["Country"] = inp["Country"].str.replace(" ","-")
countries = inp["Country"].tolist()
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False


In [11]:
countries.extend['macao','hongkong','luxemburg','lativa','elsalvador']
driver = webdriver.Chrome(executable_path="C:\\Users\PC2\\Downloads\\chromedriver_win32 (4)\\chromedriver.exe")


# Loop through each country
for country in countries:
    # Click on the country link to go to the country's page
    driver.get(f"https://wageindicator.org/salary/minimum-wage/{country}")
    driver.maximize_window()
    time.sleep(1)
    try:
    # Get the text for the "ul" element with the class name "mw__intro"
        ul_element = driver.find_element(By.XPATH,"//ul[@class='mw__intro']")
        ul_text = ul_element.text

        ul = ul_text.split("\n")
    except : 
        continue
     
    try:
        desc  = [driver.find_element(By.XPATH,"//div[@class='documentDescription description']").text]
    except:
        desc = []
    desc.extend(ul)
    df = pd.DataFrame(desc)

    # write the DataFrame to a CSV file
    df.to_csv(f'{country}.csv', header=False, index=False)
    # Find the table element for the country
    try:
            # Find the table element for the country
            
            table_element = driver.find_element(By.XPATH,"//div[@class='table-responsive']/table[@class='mw__amounts']")
            table_data = [{td[0].text.split('\n')[0]: [td[i].text if is_float(td[i].text[-1]) else '-' for i in range(1,len(td))]}  for td in [tr.find_elements(By.XPATH, ".//td") for tr in table_element.find_elements(By.XPATH, ".//tbody/tr")] ]
            d = {}
            for di in table_data:
                d.update(di)

  
            max_length = max(len(v) if isinstance(v, (list, tuple)) else 1 for v in d.values())

        # Create a new dictionary with the same length values for all keys
            new_d = {}
            for k, v in d.items():
                if isinstance(v, list):
                    if len(v) < max_length:
                        v += [None] * (max_length - len(v))
                    new_d[k] = v
                elif isinstance(v, tuple):
                    if len(v) < max_length:
                        v += (None,) * (max_length - len(v))
                    new_d[k] = list(v)  # Convert tuple to list
                else:
                    new_d[k] = [v] * max_length

        # Create a DataFrame from the new dictionary
            df = pd.DataFrame(new_d)
            df.index = [td.text for td in table_element.find_elements(By.XPATH, ".//thead/tr//td")[1:]] 
            df = df.transpose()
            with open(f'{country}.csv',mode = 'a') as file:
                df.to_csv(file)
    
    except:
        continue
    
driver.quit()

# Print the data dictionary

C:\Users\PC2\AppData\Local\Temp/ipykernel_9944/3797288609.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\\Users\PC2\\Downloads\\chromedriver_win32 (4)\\chromedriver.exe")
